In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from data_manager import *
from metadata import *

from conversion import images_to_volume

import matplotlib.pyplot as plt
%matplotlib inline

import time

Setting environment for Gordon


No vtk


In [18]:
for stack in all_nissl_stacks:

    first_sec, last_sec = metadata_cache['section_limits'][stack]

    bar = show_progress_bar(first_sec, last_sec)

    images = {}
    for sec in range(first_sec, last_sec):

        bar.value = sec

        try:
            fn = DataManager.get_image_filepath(stack, section=sec, resol='thumbnail', version='cropped_tif')
            img = img_as_ubyte(rgb2gray(imread(fn)))

            # apply foreground mask
            mask = DataManager.load_thumbnail_mask_v2(stack=stack, section=sec, version='aligned_cropped')
            img[~mask] = 0

            images[sec-1] = img
        except:
            pass

    downsample = 32
    voxel_z_size = SECTION_THICKNESS/(XY_PIXEL_DISTANCE_LOSSLESS * downsample)
    
    volume, volume_bbox = images_to_volume(images=images, voxel_size=(1, 1, voxel_z_size), first_sec=first_sec-1, last_sec=last_sec-1)
    
    basename = DataManager.get_original_volume_basename(stack=stack, volume_type='intensity')
    vol_fp = os.path.join(VOLUME_ROOTDIR, stack, basename, basename + '.bp')
    create_if_not_exists(os.path.dirname(vol_fp))

    bp.pack_ndarray_file(volume, vol_fp)
    
    bbox_fp = os.path.join(VOLUME_ROOTDIR, stack, basename, basename + '_bbox.txt')
    np.savetxt(bbox_fp, np.array(volume_bbox, np.int)[None, :], fmt='%d')